In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata
import datetime

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("jr58-6ysp", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [2]:
# Convert the appropriate columns to python datetime
results_df['week_ending'] = pd.to_datetime(results_df['week_ending'], format='%Y-%m-%dT%H:%M:%S.%f')
results_df['creation_date'] = pd.to_datetime(results_df['creation_date'], format='%Y-%m-%dT%H:%M:%S.%f')


In [3]:
# rename chare column then convert to floats 
results_df.rename(columns={'share' : 'Share (%)'}, inplace=True)
results_df['Share (%)'] = results_df['Share (%)'].astype(float) 
results_df.head(5)

,usa_or_hhsregion,week_ending,variant,Share (%),share_hi,share_lo,count_lt10,modeltype,time_interval,creation_date
0,1,2023-02-25,CH.1.1,0.002611,0.004610351752489805,0.0014449068112298846,NULL,smoothed,weekly,2023-03-10
1,1,2023-02-25,BN.1,0.000462,0.0006514294072985649,0.00032594785443507135,NULL,smoothed,weekly,2023-03-10
2,1,2023-02-25,XBB.1.5.1,0.019512,0.032935816794633865,0.011292536742985249,NULL,smoothed,weekly,2023-03-10
3,1,2023-02-25,BA.5.2.6,0.000064,0.00009090593812288716,4.413523492985405e-05,NULL,smoothed,weekly,2023-03-10
4,1,2023-02-25,BA.2.75,0.000037,0.00007894295413279906,1.6694790247129276e-05,NULL,smoothed,weekly,2023-03-10


In [4]:
results_df.tail()

usa_or_hhsregion            object
week_ending         datetime64[ns]
variant                     object
Share (%)                  float64
share_hi                    object
share_lo                    object
count_lt10                  object
modeltype                   object
time_interval               object
creation_date       datetime64[ns]
dtype: object